In [ ]:
import openai
import google.generativeai as genai
import os

# Set OpenAI API key securely
os.environ["OPENAI_API_KEY"] = "" # Your OpenAI API key
openai.api_key = os.getenv("OPENAI_API_KEY")  # Access OpenAI API key securely

# Set Google Generative AI API key
os.environ["GENAI_API_KEY"] = "" # Your Gemini API key
genai.configure(api_key=os.getenv("GENAI_API_KEY"))

def ask_gpt(prompt):
    """Interact with OpenAI's ChatGPT API using gpt-3.5-turbo."""
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )
        return response["choices"][0]["message"]["content"]
    except openai.error.OpenAIError as e:
        return f"OpenAI API error ({e.http_status}): {e}"
    except Exception as e:
        return f"Unexpected error: {e}"

def ask_gemini(prompt):
    """Interact with Google's Gemini AI."""
    try:
        # Consider checking for model compatibility based on documentation
        model = genai.GenerativeModel(model_name="gemini-1.0-pro")  # Adjust if needed
        convo = model.start_chat(history=[])
        response = convo.send_message(prompt)
        return response.text if response else "No response from Gemini."
    except Exception as e:
        return f"Error interacting with Gemini: {e}"

# Main conversation loop
print("Choose one to start the conversation:")
print("1. ChatGPT")
print("2. Gemini")

try:
    choice = int(input("Enter your choice (1 or 2): "))
    if choice == 1:
        prompt = input("Enter prompt for ChatGPT to start the conversation with Gemini: ")
        try:
            response = ask_gpt(prompt)
            print("\n\nChatGPT:\n", response)
        except openai.error.OpenAIError as e:
            print("\n\nChatGPT Error:", e)
    elif choice == 2:
        prompt = input("Enter prompt for Gemini to start the conversation with ChatGPT: ")
        try:
            response = ask_gemini(prompt)
            print("Gemini:\n", response)
        except Exception as e:
            print("Gemini Error:", e)
    else:
        print("Invalid choice. Exiting.")
        exit()

    # Alternate prompts between ChatGPT and Gemini for 10 exchanges
    for i in range(10):
        if choice == 1:
            # Only call ask_gpt if the initial choice was 1 (ChatGPT)
            if turn == "ChatGPT":
                try:
                    response = ask_gpt(response)
                    print("\n\nChatGPT:\n", response)
                except openai.error.OpenAIError as e:
                    print("ChatGPT Error:", e)
                    break
            turn = "Gemini"
        else:
            response = ask_gemini(response)
            if "error" in response.lower():
                print("Gemini Error:", response)
                break
            print("\n\nGemini:\n", response)
            turn = "ChatGPT"

    print("Goodbye!")
except ValueError:
    print("Please enter a valid number (1 or 2). Exiting.")